In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.linalg import toeplitz
from scipy.stats import zscore

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

# -------------------------------
# Funciones auxiliares
# -------------------------------

def generate_patterns(N, p):
    """Genera p patrones binarios ±1 de longitud N"""
    return np.where(np.random.rand(p, N) < 0.5, -1, 1)

def hebbian_weights(patterns):
    """Calcula matriz de pesos Hebbianos"""
    p, N = patterns.shape
    W = (patterns.T @ patterns) / N
    np.fill_diagonal(W, 0.0)
    return W

def sign_vec(x):
    """Signo con sign(0)=+1"""
    s = np.sign(x)
    s[s == 0] = 1
    return s.astype(int)

def parallel_dynamics(W, s0, max_iters=100):
    """Dinámica paralela"""
    s = s0.copy()
    for it in range(max_iters):
        s_new = sign_vec(W @ s) # actualización paralela
        if np.array_equal(s_new, s):
            return s_new, it+1
        s = s_new
    return s, max_iters

def sequential_dynamics(W, s0, max_iters=100):
    """Dinámica secuencial"""
    s = s0.copy()
    N = len(s)
    for it in range(max_iters):
        changed = False
        for i in np.random.permutation(N):
            h = W[i,:] @ s # campo local
            new_si = 1 if h >= 0 else -1
            if new_si != s[i]: # solo actualizar si cambia
                s[i] = new_si # actualizar estado
                changed = True
        if not changed:
            return s, it+1
    return s, max_iters

def overlaps(s, patterns):
    """Calcula overlaps con todos los patrones almacenados"""
    return (patterns @ s) / s.size


# -------------------------------
# Experimento principal
# -------------------------------

Ns = [500, 1000, 2000, 4000]
alphas = [0.12, 0.14, 0.16, 0.18]

max_inits = 200   # máximo número de condiciones iniciales (para controlar tiempo)
results = []

for N in Ns:
    for alpha in alphas:
        p = int(round(alpha * N))
        print(f"N={N}, alpha={alpha:.2f} -> p={p}")
        
        patterns = generate_patterns(N, p)
        W = hebbian_weights(patterns)

        if p <= max_inits:
            init_indices = np.arange(p)
        else:
            init_indices = np.random.choice(p, size=max_inits, replace=False)
        
        overlaps_parallel = []
        overlaps_sequential = []
        
        for idx in init_indices:
            s0 = patterns[idx].copy()
            # paralelo
            s_par, _ = parallel_dynamics(W, s0, max_iters=200)
            ov_par = overlaps(s_par, patterns)
            overlaps_parallel.append(ov_par)
            # secuencial
            s_seq, _ = sequential_dynamics(W, s0, max_iters=200)
            ov_seq = overlaps(s_seq, patterns)
            overlaps_sequential.append(ov_seq)
        
        overlaps_parallel = np.array(overlaps_parallel)
        overlaps_sequential = np.array(overlaps_sequential)

        diag_par = overlaps_parallel[np.arange(len(init_indices)), init_indices % p]
        diag_seq = overlaps_sequential[np.arange(len(init_indices)), init_indices % p]

        results.append({
            'N': N,
            'alpha': alpha,
            'p': p,
            'retrieval_parallel_frac': np.mean(diag_par >= 0.9),
            'retrieval_sequential_frac': np.mean(diag_seq >= 0.9),
            'mean_overlap_parallel': np.mean(diag_par),
            'mean_overlap_sequential': np.mean(diag_seq),
        })

# -------------------------------
# Resultados
# -------------------------------

summary_df = pd.DataFrame(results)
print(summary_df)

# Gráfico: fracción de recuperación vs alpha
plt.figure(figsize=(8,5))
for N in Ns:
    subset = summary_df[summary_df['N']==N].sort_values('alpha')
    plt.plot(subset['alpha'], subset['retrieval_parallel_frac'], marker='o', label=f'Paralelo N={N}')
    plt.plot(subset['alpha'], subset['retrieval_sequential_frac'], marker='x', linestyle='--', label=f'Secuencial N={N}')
plt.xlabel('alpha = p/N')
plt.ylabel('Fracción de recuperación (overlap ≥ 0.9)')
plt.title('Capacidad efectiva del Hopfield')
plt.legend()
plt.show()